# Bunch commands for pyTao

In [ ]:
from pytao import Tao, util
from pytao.util import parse_bool
import os

In [ ]:
# Point to local installation
BASE_DIR=os.environ['ACC_ROOT_DIR'] 
print('Bmad installation: ', BASE_DIR)


# Pick an example init
root = BASE_DIR+'tao/examples/csr_beam_tracking/'
os.chdir(root)
init = root+'tao.init'
# Make tao instance
tao=Tao()   
tao.init('-noplot -init '+init);

---
# bunch1

In [ ]:
# Bunch data for universe 1, branch 0, element 0 (beginnig) for the model.
tao.cmd('python bunch1 1@0>>0|model 1')

In [ ]:
%%tao
sho beam 0

In [ ]:
# Parse output into dict
bdata = util.parse_tao_python_data(tao.cmd('python bunch1 1@0>>0|model 1'))
bdata.keys()

In [ ]:
bdata['beam_saved']

In [ ]:
xdat = tao.cmd_real('python bunch1 1@0>>8|model x')
pxdat = tao.cmd_real('python bunch1 1@0>>8|model px')
ydat = tao.cmd_real('python bunch1 1@0>>8|model y')
pydat = tao.cmd_real('python bunch1 1@0>>8|model py')
zdat = tao.cmd_real('python bunch1 1@0>>8|model z')
pzdat = tao.cmd_real('python bunch1 1@0>>8|model pz')
chargedat = tao.cmd_real('python bunch1 1@0>>8|model charge')
p0cdat = tao.cmd_real('python bunch1 1@0>>8|model p0c')
state = tao.cmd_integer('python bunch1 1@0>>8|model state') # integers
# Unique p0C (eV), charge (pC)
set(p0cdat), sum(chargedat)*1e12

# Plot in matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
xdata = 1000*xdat
ydata = 1000*pxdat
weights = chargedat

In [ ]:
mycmap = plt.get_cmap('plasma') # viridis plasma inferno magma and _r versions
mycmap.set_under(color='white') # map 0 to this color
myvmin = 1e-30 # something tiny
# Bin particles
plt.hist2d(x=1000*xdata, y=ydata,  bins=2*[40], weights=weights, cmap=mycmap, vmin=myvmin)
plt.xlabel('x (mm)')
plt.ylabel('px (mrad)')
plt.show()

# Plot in Bokeh

In [ ]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh import palettes, colors
from bokeh.models import ColumnDataSource, HoverTool
output_notebook(verbose=False, hide_banner=True)

pal = palettes.Viridis[256]
white=colors.named.white
pal[0] = white # replace 0 with white

In [ ]:
xdata = 1000*zdat
ydata = 1000*pzdat

In [ ]:
H, xedges, yedges = np.histogram2d(xdata, ydata, weights=chargedat, bins=40)
xmin, xmax = min(xedges), max(xedges)
ymin, ymax = min(yedges), max(yedges)

In [ ]:
ds = ColumnDataSource(data=dict(image=[H.transpose()]))   
p = figure(x_range=[xmin, xmax], y_range=[ymin, ymax], title='Bunch at end', 
           x_axis_label='z (mm)', y_axis_label='pz (1e-3)',
           plot_width=500, plot_height=500)
p.image(image='image', source=ds, 
    x=xmin, y=ymin, dw=xmax-xmin, dh=ymax-ymin, palette=pal)
show(p)